In [2]:
# Needed modules
import pandas as pd
import os
import subprocess

In [3]:
# Importing data needed
data = pd.read_csv("./data/Results_Program_09-05-24_32ChrSIDERs.csv", sep=",", header=0)
print(data.shape)
print(data.dtypes)
data.head()

(5972, 14)
qseqid      float64
sseqid       object
pident      float64
length        int64
qstart      float64
qend        float64
sstart        int64
send          int64
evalue      float64
bitscore    float64
qlen        float64
slen        float64
sstrand      object
sseq         object
dtype: object


,qseqid,sseqid,pident,length,qstart,qend,sstart,send,evalue,bitscore,qlen,slen,sstrand,sseq
0,NaN,LinJ.01,NaN,1000,NaN,NaN,1000,1,NaN,NaN,NaN,NaN,minus,GTTCTATCCATCGACCTGCACCTGCACACATGAGCTGCAAAAAGTT...
1,NaN,LinJ.01,NaN,1000,NaN,NaN,6881,5882,NaN,NaN,NaN,NaN,minus,GTGGCGATACGTGCGTTACCTCGGCTACGGCACGCTCTACCAGTCC...
2,NaN,LinJ.01,NaN,1000,NaN,NaN,14301,13302,NaN,NaN,NaN,NaN,minus,CGTCTTCTCTCCGCATCCCTCCATGCCCCCTCCTCCATCTCCTCCC...
3,NaN,LinJ.01,NaN,1000,NaN,NaN,17384,16385,NaN,NaN,NaN,NaN,minus,CTCGTCCGCCTCCGACCGACACCCCGCCCCTGGGCCACGACATTCA...
4,NaN,LinJ.01,NaN,669,NaN,NaN,24867,24199,NaN,NaN,NaN,NaN,minus,GAGAGCGGTGTGGCGCATGGTGCCGCGCCCTTTGCGCAGGCGGAGC...


In [4]:
# Check coor. in "minus" strand, where start > end
data[data["sstrand"] == "minus"].head()

,qseqid,sseqid,pident,length,qstart,qend,sstart,send,evalue,bitscore,qlen,slen,sstrand,sseq
0,NaN,LinJ.01,NaN,1000,NaN,NaN,1000,1,NaN,NaN,NaN,NaN,minus,GTTCTATCCATCGACCTGCACCTGCACACATGAGCTGCAAAAAGTT...
1,NaN,LinJ.01,NaN,1000,NaN,NaN,6881,5882,NaN,NaN,NaN,NaN,minus,GTGGCGATACGTGCGTTACCTCGGCTACGGCACGCTCTACCAGTCC...
2,NaN,LinJ.01,NaN,1000,NaN,NaN,14301,13302,NaN,NaN,NaN,NaN,minus,CGTCTTCTCTCCGCATCCCTCCATGCCCCCTCCTCCATCTCCTCCC...
3,NaN,LinJ.01,NaN,1000,NaN,NaN,17384,16385,NaN,NaN,NaN,NaN,minus,CTCGTCCGCCTCCGACCGACACCCCGCCCCTGGGCCACGACATTCA...
4,NaN,LinJ.01,NaN,669,NaN,NaN,24867,24199,NaN,NaN,NaN,NaN,minus,GAGAGCGGTGTGGCGCATGGTGCCGCGCCCTTTGCGCAGGCGGAGC...


In [5]:
# Change coor so that start < end
data.loc[data['sstrand'] == 'minus', ['sstart', 'send']] = data.loc[data['sstrand'] == 'minus', ['send', 'sstart']].values

In [6]:
# Check if it got fixed
data[data["sstrand"] == "minus"].head()

,qseqid,sseqid,pident,length,qstart,qend,sstart,send,evalue,bitscore,qlen,slen,sstrand,sseq
0,NaN,LinJ.01,NaN,1000,NaN,NaN,1,1000,NaN,NaN,NaN,NaN,minus,GTTCTATCCATCGACCTGCACCTGCACACATGAGCTGCAAAAAGTT...
1,NaN,LinJ.01,NaN,1000,NaN,NaN,5882,6881,NaN,NaN,NaN,NaN,minus,GTGGCGATACGTGCGTTACCTCGGCTACGGCACGCTCTACCAGTCC...
2,NaN,LinJ.01,NaN,1000,NaN,NaN,13302,14301,NaN,NaN,NaN,NaN,minus,CGTCTTCTCTCCGCATCCCTCCATGCCCCCTCCTCCATCTCCTCCC...
3,NaN,LinJ.01,NaN,1000,NaN,NaN,16385,17384,NaN,NaN,NaN,NaN,minus,CTCGTCCGCCTCCGACCGACACCCCGCCCCTGGGCCACGACATTCA...
4,NaN,LinJ.01,NaN,669,NaN,NaN,24199,24867,NaN,NaN,NaN,NaN,minus,GAGAGCGGTGTGGCGCATGGTGCCGCGCCCTTTGCGCAGGCGGAGC...


In [7]:
# Check if every "sstart" is < "data"
## Sum all true values
print(f"""
From {data.shape[0]} rows, there are {sum(data["sstart"] < data["send"])} rows where "sstart" < "send"
      """)


From 5972 rows, there are 5972 rows where "sstart" < "send"
      


In [8]:
# Take only what's needed
data_2 = data[["sseqid", "sstart", "send"]].copy()

In [11]:
# path to bedops files and .bed file creation
path_all_elements_bed = "./bedops_files/32chr_all_elements.bed"
data_2.to_csv(path_all_elements_bed, sep="\t", header=False, index=False)

In [12]:
# bedops call
cmd = f"bedops --merge {path_all_elements_bed}"
merged_data = subprocess.check_output(cmd, shell=True, universal_newlines=True)
merged_df = pd.DataFrame([x.split("\t") for x in merged_data.split("\n") if x],
                         columns=["sseqid", "sstart", "send"])
merged_df[['sstart', 'send']] = merged_df[['sstart', 'send']].apply(pd.to_numeric)
print(f"""
From {data_2.shape[0]} rows, there are {merged_df.shape[0]} rows after merging
      """)


From 5972 rows, there are 3533 rows after merging
      


In [14]:
merged_df.head(10)

,sseqid,sstart,send
0,LinJ.01,1,1000
1,LinJ.01,5882,6881
2,LinJ.01,13302,14301
3,LinJ.01,16385,17384
4,LinJ.01,24199,24867
5,LinJ.01,32788,33787
6,LinJ.01,35374,36299
7,LinJ.01,39790,40917
8,LinJ.01,54986,55911
9,LinJ.01,68445,69444


In [15]:
# Now let's get the sequences
def get_data_sequence(data, strand, genome_fasta):
    sequences = []
    for _, row in data.iterrows():
        sseqid = row["sseqid"]
        start = row["sstart"]
        end = row["send"]
        cmd = f"blastdbcmd -db {genome_fasta} -entry {sseqid} -range {start}-{end} -strand {strand} -outfmt %s"

        sequence = subprocess.check_output(cmd, shell=True, universal_newlines=True).replace('\n', '')

        sequences.append({
            "sseqid": sseqid,
            "sstart": start,
            "send": end,
            "sstrand": strand,
            "sseq": sequence
        })

    sequences_df = pd.DataFrame(sequences)
    return sequences_df


In [16]:
# Prepare dict creation
def blastn_dic(path_input, path_output):
    # "parse_seqids" is used to keep the sequence ID in the output.
    cmd = f"makeblastdb -in {path_input} -dbtype nucl -parse_seqids -out {path_output}"
    subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

In [13]:
blastn_dic(path_input="./data/dict/TriTrypDB-67_LinfantumJPCM5_Genome.fasta",
            path_output="./data/dict/TriTrypDB-67_LinfantumJPCM5_Genome.fasta")

In [17]:
# Get sequences
data_after_merge = get_data_sequence(data=merged_df, 
                                     strand="plus", 
                                     genome_fasta="./data/dict/TriTrypDB-67_LinfantumJPCM5_Genome.fasta")
print(f"""
From {merged_df.shape[0]} rows, there are {data_after_merge.shape[0]} rows with sequences
      """)


From 3533 rows, there are 3533 rows with sequences
      


In [18]:
# Check data
print(data_after_merge.shape)
print(data_after_merge.dtypes)
data_after_merge.head()

(3533, 5)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
dtype: object


,sseqid,sstart,send,sstrand,sseq
0,LinJ.01,1,1000,plus,ACACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCA...
1,LinJ.01,5882,6881,plus,AGAAGGCAGTTTGTGGTTAGGAAAAAGTTGCGCCTCGTTCACCTCT...
2,LinJ.01,13302,14301,plus,GTCGACCGTGCAAGAGGAACTCAACACCGCTCTACTCGGCGTGTCA...
3,LinJ.01,16385,17384,plus,CTCCTTGCACATGTCGTAGTACTCCACGCACTGATTGTCATTGTAC...
4,LinJ.01,24199,24867,plus,CCCCCTCACCCTCTATCCCTGCCAACGCCGAACCACTTCTGGTGCT...


In [19]:
# inset length column
length_column = data_after_merge["sseq"].apply(len)
data_after_merge.insert(1, "length", length_column)
data_after_merge.head()

,sseqid,length,sstart,send,sstrand,sseq
0,LinJ.01,1000,1,1000,plus,ACACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCA...
1,LinJ.01,1000,5882,6881,plus,AGAAGGCAGTTTGTGGTTAGGAAAAAGTTGCGCCTCGTTCACCTCT...
2,LinJ.01,1000,13302,14301,plus,GTCGACCGTGCAAGAGGAACTCAACACCGCTCTACTCGGCGTGTCA...
3,LinJ.01,1000,16385,17384,plus,CTCCTTGCACATGTCGTAGTACTCCACGCACTGATTGTCATTGTAC...
4,LinJ.01,669,24199,24867,plus,CCCCCTCACCCTCTATCCCTGCCAACGCCGAACCACTTCTGGTGCT...


In [20]:
data_after_merge.to_csv("./data/3.Merged_data_32chr.csv", sep=",", index=False)